# Lecture 10-1

# Messing around with LLMs with Haystack

Step 0: Get OpenAI API Key

To get an OpenAI API key, you need to sign up for an OpenAI account and subscribe to their API service. 

- Go to the OpenAI developer website. https://platform.openai.com/
- Click on "Sign Up" to create a new account, or "Log In" if you already have an account.
- Click on the Settings Gear. 
- Navigate to "API Keys" 
- You may need to set up a billing plan. If you stick with gpt 3.5, it is cheap. You can buy $5 of credits, which will be more than enough for experimentation.
- Go to the API keys section of your OpenAI account.
- Click on "Create new secret key" to generate a new API key.
- Copy the generated API key.
- Create a `.env` file in your project directory.
- In the `.env` file, enter:

```
OPENAI_API_KEY=text_of_super_secret_api_key
```

- You won't be able to see the api key again once you close the dialog. If you don't store it and lose it, you'll need to create a new API key.  

Step 1: Create a New Virtual Environment

- Open your terminal
- Create a new virtual environment and activate it:

```
python -m venv haystackai
haystackai\Scripts\activate
```

just remember to deactivate your virtual environment at the end

```
deactivate
```


See: https://haystack.deepset.ai/tutorials/27_first_rag_pipeline

Once you've activated the haystack environment, install Jupyter, Haystack, and other necessary packages in your Conda environment:

```
pip install openai python-dotenv
pip install torch
pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=2.2.0"
```

Open the notebook

In [1]:
# cell 1: let's haystack know you are running a tutorial
from haystack.telemetry import tutorial_running

tutorial_running(27)


In [2]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()


In [3]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]


In [4]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()


In [5]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)


In [8]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)


In [9]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-5-nano")


In [10]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [11]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

A colossal bronze statue of Helios the sun god, about 33 meters tall, standing on a white marble pedestal near Rhodes’ harbor. It was built with an iron frame and brass plates for the skin, and the head is described as having curly hair with radiating sun-ray spikes. Some reliefs suggest it may have been shown in a pose akin to shielding the eyes from the sun.


In [12]:
question = "When did construction for the Rhodes statue begin?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

292 BC.


In [13]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]


In [14]:
for question in examples:
    print(question)
    response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
    print(response["llm"]["replies"][0])


Where is Gardens of Babylon?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The exact location is not definitively known. Traditionally they’re said to be in Babylon (near modern Hillah, Iraq), but there’s no archaeological evidence, and some scholars suggest they were in Nineveh instead.
Why did people build Great Pyramid of Giza?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

It was built as the tomb of Khufu (Cheops), the Fourth Dynasty pharaoh, as a monumental funerary monument within the Giza pyramid complex.
What does Rhodes Statue look like?


...

- It is one of the Seven Wonders of the Ancient World, marking it as a symbol of ancient architectural and artistic achievement.
- Built around 292–280 BC to celebrate Rhodes’ successful defense, it demonstrated remarkable engineering, using iron tie bars with bronze skin and a marble pedestal.
- At about 33 meters tall, it was among the tallest statues of the ancient world and became a powerful emblem of Rhodian ingenuity and devotion to Helios.
- It stood for a time as a cultural and historical symbol, influencing later literature and art, and shaping myths about its location and appearance (including the famous but inaccurate idea that it straddled the harbor).
- Destroyed by an earthquake in 226 BC and not rebuilt, its legacy persists in historical study and modern imagination as a quintessential ancient Wonder.
What happened to the Tomb of Mausolus?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

It was destroyed and dismantled. A series of earthquakes from the medieval period shattered the tomb, and its stones were salvaged and used to fortify Bodrum Castle (and elsewhere). By about the 15th–16th centuries much of the tomb had been broken up, with only the base remaining, and today only ruins and fragments survive.
How did Colossus of Rhodes collapse?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

It collapsed in the 226 BC Rhodes earthquake, which snapped its knees and caused the statue to fall to the ground.


In [15]:
question = "What is Miles's favorite color?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

There is no information about a person named Miles or his favorite color in the provided text.
